In [21]:
import pandas as pd

df = pd.read_excel(r"C:\Users\user\Desktop\df.xlsx")

df.head()

,남자사람_10_남_00193.jpg,남자사람_10_남_00979.jpg,남자사람_10_남_01252.jpg,남자사람_10_남_01461.jpg,남자사람_10_남_01718.jpg,남자사람_10_남_01745.jpg,남자사람_10_남_01841.jpg,남자사람_10_남_02184.jpg,남자사람_10_남_02379.jpg,남자사람_10_남_03153.jpg,...,남자사람_8_여_12553.jpg,남자사람_8_여_12815.jpg,남자사람_8_여_12899.jpg,남자사람_8_여_13112.jpg,남자사람_8_여_13439.jpg,남자사람_9_남_00478.jpg,남자사람_9_남_00831.jpg,남자사람_9_남_00934.jpg,남자사람_9_남_00964.jpg,남자사람_9_남_01030.jpg
0,5,5,6,6,5,5,5,6,5,5,...,6,5,5,5,5,5,5,5,5,5


In [22]:
ser = df.iloc[0]

ser.iloc[ser.values>5].index

Index(['남자사람_10_남_01252.jpg', '남자사람_10_남_01461.jpg', '남자사람_10_남_02184.jpg',
       '남자사람_10_남_03586.jpg', '남자사람_10_남_03684.jpg', '남자사람_10_남_07508.jpg',
       '남자사람_10_남_08201.jpg', '남자사람_10_여_06932.jpg', '남자사람_10_여_10823.jpg',
       '남자사람_10_여_11762.jpg', '남자사람_11_남_01241.jpg', '남자사람_11_남_05750.jpg',
       '남자사람_11_남_07332.jpg', '남자사람_11_남_09034.jpg', '남자사람_11_여_05585.jpg',
       '남자사람_11_여_06326.jpg', '남자사람_11_여_08819.jpg', '남자사람_12_남_02447.jpg',
       '남자사람_12_남_03180.jpg', '남자사람_12_남_07015.jpg', '남자사람_12_남_12742.jpg',
       '남자사람_12_여_00842.jpg', '남자사람_12_여_01131.jpg', '남자사람_12_여_02654.jpg',
       '남자사람_12_여_03616.jpg', '남자사람_12_여_03742.jpg', '남자사람_12_여_03787.jpg',
       '남자사람_12_여_04196.jpg', '남자사람_12_여_04456.jpg', '남자사람_12_여_10034.jpg',
       '남자사람_13_남_04105.jpg', '남자사람_13_남_06544.jpg', '남자사람_13_남_07639.jpg',
       '남자사람_13_남_10272.jpg', '남자사람_13_남_10317.jpg', '남자사람_13_남_12378.jpg',
       '남자사람_13_남_12481.jpg', '남자사람_13_여_01598.jpg', '남자사람_13_여_01917.jpg',
       '남자사람

In [25]:
import os
import numpy as np
import cv2
import json
import math

folder_path1 = r"C:\Users\user\Desktop\23_kdt_hackathon\hackathon_data\data\Training\01.원천데이터\TS_남자사람"
folder_path2 = r"C:\Users\user\Desktop\23_kdt_hackathon\hackathon_data\data\Training\02.라벨링데이터\TL_남자사람"


for file_name in ser.iloc[ser.values>5].index:
    file_path = os.path.join(folder_path1, file_name)
    json_path = os.path.join(folder_path2, file_name)
    try:
        img_array = np.fromfile(file_path, np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        json_path = json_path[:-2]+'son'
        
        with open(json_path, "r", encoding="utf-8") as file:
            json_data = json.load(file)
            
            # 얼굴에서 이목구비 지우기
            for box in json_data['annotations']['bbox']:    
                if (box['label']=='얼굴' or box['label']=='머리' or box['label']=='사람전체') == False:
                    x, y, width, height = box['x'], box['y'], box['w'], box['h']  #좌표
                    background_color = (255, 255, 255)
                    image[y:y+height, x:x+width] = background_color

            face = json_data['annotations']['bbox'][2]
            if face['label'] != '얼굴':
                continue
            x, y, width, height = face['x'], face['y'], face['w'], face['h']  # 얼굴좌표
            
            # 이목구비 지워진 얼굴이 roi에 저장
            roi = image[y:y+height, x:x+width]
            
            # 노이즈 제거
            roi = cv2.medianBlur(roi, ksize=5)
            
            # 상단 절반을 흰색으로 채우기
            top_height = int(0.75 * height)
            roi[:top_height, :] = (255, 255, 255)
            
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            
            # Harris 코너 검출을 위한 파라미터 설정
            block_size = 6
            ksize = 3
            k = 0.05

            # Harris 코너 검출
            corners = cv2.cornerHarris(roi, block_size, ksize, k)

            # 결과 이미지에서 좌표를 찾기 위한 임계값 설정
            threshold = 0.05* corners.max()

            # 코너 위치 표시를 위한 이미지 복사
            corner_image = np.copy(roi)

            # 코너 위치를 얻기 위한 조건에 맞는 포인트 추출
            corner_points = np.argwhere(corners > threshold)
            

            # 제거할 거리 기준
            threshold_distance = 30

            # 일정 거리 이내의 좌표를 제외한 좌표를 저장할 리스트
            filtered_coord_list = []

            # 거리 계산 함수
            def distance(p1, p2):
                x1, y1 = p1
                x2, y2 = p2
                return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

            # 좌표 간의 거리를 검사하고 일정 거리 이내의 좌표를 제외
            for i, point1 in enumerate(corner_points):
                within_threshold = False
                for j, point2 in enumerate(corner_points):
                    if (i < j) and (distance(point1, point2) <= threshold_distance):
                        within_threshold = True
                        break
                if not within_threshold:
                    filtered_coord_list.append(point1)

            # # 결과 출력
            # print(filtered_coord_list)

            
            
            # 코너 위치에 원 그리기 (빨간색, 두께 1)
            for point in filtered_coord_list:
                x, y = point[1], point[0]
                cv2.circle(corner_image, (x, y), 3, (255, 0, 0), -1)
            
            
            
        output_path = os.path.join(r"C:\Users\user\Desktop\23_kdt_hackathon\hackathon_data\data\Training\01.원천데이터\TS_남자사람_각진얼굴", file_name)
                    
        # 한글포함 경로에 넣기 위함..
        ret, img_arr = cv2.imencode('.jpg', corner_image)

        if ret:
            with open(output_path, mode='w+b') as f:
                img_arr.tofile(f)
                
    except Exception as e:
        print(f"파일 읽기 및 JSON 파싱 오류: {file_name}, 오류 메시지: {str(e)}")